In [1]:
from CONSTANT import *
from tools import *
import pandas as pd
import numpy as np
from torch.utils.data import (TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)

from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoderLayer, TransformerDecoder

from models import *

import torch

from torch import nn
import torch.nn.functional as F
from tqdm import tqdm


use_cuda = True
valid='loso'

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('using', device)

from representation.SigRepre import MultiSignalRepresentation

using cuda


In [2]:
class Params(object):
    def __init__(self, lr=0.001, epoch=100, valid='cv', target='valence', batch_size=16, use_cuda=use_cuda):
        self.batch_size = batch_size
        self.valid = valid
        self.target = target
        self.epoch = epoch
        self.lr = lr
        self.metrics_dict = {}
        self.debug = False
        self.out_channels = 32
        self.out_channels = 32
        self.hidden_size = 64
        self.num_layers = 1
        self.fcn_input = 50432
        self.dropout=0.2
        self.use_cuda=use_cuda
        self.device=torch.device(
            'cuda' if torch.cuda.is_available() and self.use_cuda else 'cpu')

In [12]:
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error

In [5]:
args = Params()

model = MultiSignalRepresentation(output_size=40, device=args.device)
model = model.to(args.device)
model.load_state_dict(torch.load(r'./output/0.0001_256_maskp0.8_checkpoint.pt'))

<All keys matched successfully>

In [7]:
def encoder(data):
    encoded_signals = []

    for i, (x, y) in tqdm(enumerate(data)):
        encodedx = model.encoder(x).flatten(start_dim=1).cpu().detach().numpy()
        encoded_signals.append(encodedx)
    return np.concatenate(encoded_signals, axis=0)

In [14]:
spliter = load_model(r'./processed_signal/HKU956/last15_400_4s_step_2s_spliter10.pkl')
data = pd.read_pickle(r'./processed_signal/HKU956/last15_400_4s_step_2s.pkl')
# data = pd.read_csv(r'./processed_signal/HKU956/400_4s_step_2s.csv')

all_res = []

for i, k in enumerate(spliter[valid]):
    print('Fold', i)
    train_index = k['train_index']
    test_index = k['test_index']
    dataprepare = DataPrepare(args=args, target='valence_rating', data=data, train_index=train_index, test_index=test_index, device=device, batch_size=4)
    ytest = dataprepare.ytest
    ytrain = dataprepare.ytrain
    train_data, test_data = dataprepare.get_data()

    X_train = encoder(train_data)
    X_test = encoder(test_data)
    y_train = ytrain.reshape(-1, ).cpu().detach().numpy()
    y_test = ytest.reshape(-1, ).cpu().detach().numpy()

    # clf = LGBMClassifier()
    clf = LGBMRegressor()
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(np.sqrt(mean_squared_error(y_test, pred)))
    # results = classification_report(y_test, pred, output_dict=True)
    # print(results['accuracy'])
    # all_res.append(results)

Fold 0
(5412, 4, 400) (5412, 1) (324, 4, 400) (324, 1)


1353it [01:29, 15.11it/s]
81it [00:05, 16.07it/s]


4.012040563342592
Fold 1
(5154, 4, 400) (5154, 1) (582, 4, 400) (582, 1)


1289it [01:20, 16.10it/s]
146it [00:08, 16.44it/s]


4.497242710822926
Fold 2
(5232, 4, 400) (5232, 1) (504, 4, 400) (504, 1)


...

4.809018828922758
Fold 8
(5262, 4, 400) (5262, 1) (474, 4, 400) (474, 1)


1316it [01:20, 16.37it/s]
119it [00:07, 16.25it/s]


2.8992933553344797
Fold 9
(5280, 4, 400) (5280, 1) (456, 4, 400) (456, 1)


1320it [01:22, 16.09it/s]
114it [00:07, 15.01it/s]


4.854482256502285


In [10]:
all_res[5]

{'0.0': {'precision': 0.4351464435146444,
  'recall': 0.48148148148148145,
  'f1-score': 0.4571428571428572,
  'support': 216},
 '1.0': {'precision': 0.5773584905660377,
  'recall': 0.53125,
  'f1-score': 0.5533453887884267,
  'support': 288},
 'accuracy': 0.5099206349206349,
 'macro avg': {'precision': 0.506252467040341,
  'recall': 0.5063657407407407,
  'f1-score': 0.5052441229656419,
  'support': 504},
 'weighted avg': {'precision': 0.5164104704011548,
  'recall': 0.5099206349206349,
  'f1-score': 0.5121157323688968,
  'support': 504}}

In [12]:
args = Params()
dataprepare = DataPrepare(args=args, target='valence', data=data, train_index=train_index, test_index=test_index, device=device, batch_size=4)
# train_dataloader, test_dataloader = dataprepare.get_data()

(5280, 4, 400) (5280, 1) (456, 4, 400) (456, 1)


In [13]:
xtest = dataprepare.xtest
xtrain = dataprepare.xtrain
ytest = dataprepare.ytest
ytrain = dataprepare.ytrain
xtest.shape, xtrain.shape, ytest.shape, ytrain.shape

(torch.Size([456, 4, 400]),
 torch.Size([5280, 4, 400]),
 torch.Size([456, 1]),
 torch.Size([5280, 1]))

In [25]:
from models import MER

In [14]:
model.output_layer = MER.MERClassifer(args, 2)

In [15]:
model = model.to(args.device)

In [16]:
model(xtest[:5]).shape

torch.Size([5, 2])

In [15]:
data = load_model(r'./processed_signal/HKU956/400_4s_step_2s.pkl')

In [20]:
data['valence_rating'].value_counts()

 0.0    790
 6.2    524
 1.4    523
 1.8    479
 2.2    434
       ... 
-2.5     14
-7.8     14
-4.6     14
 7.7     14
-1.6     13
Name: valence_rating, Length: 185, dtype: int64

In [ ]:
# encoded_xtrain = model.encoder(xtrain).flatten(start_dim=1)
# encoded_xtrain.shape

1353it [01:27, 15.40it/s]
81it [00:05, 14.70it/s]


{'0.0': {'precision': 0.4,
  'recall': 0.0625,
  'f1-score': 0.10810810810810811,
  'support': 96},
 '1.0': {'precision': 0.7087378640776699,
  'recall': 0.9605263157894737,
  'f1-score': 0.8156424581005587,
  'support': 228},
 'accuracy': 0.6944444444444444,
 'macro avg': {'precision': 0.554368932038835,
  'recall': 0.5115131578947368,
  'f1-score': 0.4618752831043334,
  'support': 324},
 'weighted avg': {'precision': 0.617259978425027,
  'recall': 0.6944444444444444,
  'f1-score': 0.6060026506953882,
  'support': 324}}

In [50]:
data = load_model(r'./processed_signal/HKU956/last15_400_4s_step_2s.pkl')
data['valence_label'].value_counts()

1    2934
0    2802
Name: valence_label, dtype: int64

1    2934
0    2802
Name: valence_label, dtype: int64